In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg

import os.path as op
import time


from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras_tqdm import TQDMNotebookCallback
import numpy as np

from kiki import kiki_net
from data import MaskedUntouched2DSequence
from evaluate import psnr

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

n_samples_train = 7135
n_samples_val = 7135

n_volumes_train = 199
n_volumes_val = 199

In [4]:
# generators
AF = 4
# MaskShifted2DSequence, MaskShiftedSingleImage2DSequence, MaskedUntouched2DSequence
train_gen = MaskedUntouched2DSequence(train_path, af=AF, inner_slices=8)
val_gen = MaskedUntouched2DSequence(val_path, af=AF)

In [5]:
run_params = {
    'n_cascade': 5, 
    'n_convs': 5, 
    'n_filters': 32,
    'noiseless': True,
}

n_epochs = 10
run_id = f'cascadenet_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

cascadenet_af4_1566809696


In [6]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=2, save_weights_only=True)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
lr_on_plat_cback = ReduceLROnPlateau(monitor='val_keras_psnr', min_lr=5*1e-5, mode='auto', patience=3)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")

In [7]:
model = kiki_net(lr=1e-3, **run_params)
print(model.summary(line_length=100))

W0826 10:54:56.743852 140456916723456 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 10:54:56.763682 140456916723456 deprecation.py:323] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/pdnet_crop.py:63: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0826 10:54:56.800102 140456916723456 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 10:54:56.807281 140456916723456 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:4467: The name tf.truncated_normal is deprecated. Please use tf.ra

Model: "model_1"
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
kspace_input (InputLayer)        (None, 640, None, 1)  0                                            
____________________________________________________________________________________________________
ifft_simple (Lambda)             (None, 640, None, 1)  0           kspace_input[0][0]               
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 640, None, 1)  0           ifft_simple[0][0]                
____________________________________________________________________________________________________
lambda_2 (Lambda)                (None, 640, None, 1)  0           ifft_simple[0][0]                
__________________________________________________________________________

In [8]:
# simple overfit trials
# lr_on_plat_cback = ReduceLROnPlateau(monitor='keras_psnr', min_lr=5*1e-5, mode='max', patience=5, min_delta=0.01)

data = train_gen[0]
val_data = val_gen[1]
model.fit(
    x=data[0], 
    y=data[1], 
    validation_data=val_data, 
    batch_size=data[0][0].shape[0], 
    callbacks=[tboard_cback],
    epochs=100,
    verbose=2, 
    shuffle=False,
)

W0826 10:54:59.330177 140456916723456 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:1021: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0826 10:54:59.932175 140456916723456 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:1008: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 8 samples, validate on 38 samples


W0826 10:55:02.580454 140456916723456 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/callbacks.py:1122: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0826 10:55:02.582576 140456916723456 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/callbacks.py:1125: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/100
 - 10s - loss: 1.3345e-05 - mean_squared_error: 3.2274e-10 - keras_psnr: 33.5207 - keras_ssim: 0.7400 - val_loss: 1.3010e-05 - val_mean_squared_error: 2.9963e-10 - val_keras_psnr: 18.5789 - val_keras_ssim: 0.1867
Epoch 2/100
 - 5s - loss: 1.0623e-05 - mean_squared_error: 2.0439e-10 - keras_psnr: 35.5041 - keras_ssim: 0.8084 - val_loss: 1.1609e-05 - val_mean_squared_error: 2.2879e-10 - val_keras_psnr: 19.5954 - val_keras_ssim: 0.2264
Epoch 3/100
 - 4s - loss: 8.7864e-06 - mean_squared_error: 1.4016e-10 - keras_psnr: 37.1425 - keras_ssim: 0.8589 - val_loss: 1.0720e-05 - val_mean_squared_error: 1.8965e-10 - val_keras_psnr: 20.3076 - val_keras_ssim: 0.2624
Epoch 4/100
 - 4s - loss: 7.7165e-06 - mean_squared_error: 1.0808e-10 - keras_psnr: 38.2712 - keras_ssim: 0.8877 - val_loss: 1.0241e-05 - val_mean_squared_error: 1.7076e-10 - val_keras_psnr: 20.7153 - val_keras_ssim: 0.2873
Epoch 5/100
 - 5s - loss: 7.1930e-06 - mean_squared_error: 9.4055e-11 - keras_psnr: 38.8746 - keras_ssi

Epoch 38/100
 - 5s - loss: 6.3008e-06 - mean_squared_error: 7.2931e-11 - keras_psnr: 39.9795 - keras_ssim: 0.9170 - val_loss: 9.5034e-06 - val_mean_squared_error: 1.4452e-10 - val_keras_psnr: 21.3804 - val_keras_ssim: 0.3591
Epoch 39/100
 - 5s - loss: 6.3003e-06 - mean_squared_error: 7.2904e-11 - keras_psnr: 39.9811 - keras_ssim: 0.9170 - val_loss: 9.5037e-06 - val_mean_squared_error: 1.4453e-10 - val_keras_psnr: 21.3802 - val_keras_ssim: 0.3591
Epoch 40/100
 - 5s - loss: 6.2998e-06 - mean_squared_error: 7.2876e-11 - keras_psnr: 39.9828 - keras_ssim: 0.9170 - val_loss: 9.5040e-06 - val_mean_squared_error: 1.4453e-10 - val_keras_psnr: 21.3800 - val_keras_ssim: 0.3591
Epoch 41/100
 - 5s - loss: 6.2993e-06 - mean_squared_error: 7.2849e-11 - keras_psnr: 39.9844 - keras_ssim: 0.9170 - val_loss: 9.5042e-06 - val_mean_squared_error: 1.4454e-10 - val_keras_psnr: 21.3798 - val_keras_ssim: 0.3592
Epoch 42/100
 - 5s - loss: 6.2988e-06 - mean_squared_error: 7.2821e-11 - keras_psnr: 39.9860 - keras

Epoch 75/100
 - 5s - loss: 6.2799e-06 - mean_squared_error: 7.1820e-11 - keras_psnr: 40.0459 - keras_ssim: 0.9174 - val_loss: 9.4988e-06 - val_mean_squared_error: 1.4442e-10 - val_keras_psnr: 21.3835 - val_keras_ssim: 0.3587
Epoch 76/100
 - 5s - loss: 6.2790e-06 - mean_squared_error: 7.1763e-11 - keras_psnr: 40.0494 - keras_ssim: 0.9174 - val_loss: 9.4985e-06 - val_mean_squared_error: 1.4442e-10 - val_keras_psnr: 21.3837 - val_keras_ssim: 0.3587
Epoch 77/100
 - 5s - loss: 6.2780e-06 - mean_squared_error: 7.1698e-11 - keras_psnr: 40.0533 - keras_ssim: 0.9174 - val_loss: 9.4983e-06 - val_mean_squared_error: 1.4442e-10 - val_keras_psnr: 21.3838 - val_keras_ssim: 0.3586
Epoch 78/100
 - 5s - loss: 6.2769e-06 - mean_squared_error: 7.1630e-11 - keras_psnr: 40.0574 - keras_ssim: 0.9174 - val_loss: 9.4980e-06 - val_mean_squared_error: 1.4441e-10 - val_keras_psnr: 21.3840 - val_keras_ssim: 0.3586
Epoch 79/100
 - 4s - loss: 6.2757e-06 - mean_squared_error: 7.1559e-11 - keras_psnr: 40.0617 - keras